In [ ]:
Задача
В вашем распоряжении имеется уже предобработанный и очищенный датасет с фактами покупок абонентами телепередач от компании 
E-Contenta. По доступным вам данным, нужно предсказать вероятность покупки других передач этими, а, возможно, и другими 
абонентами. При решении задачи запрещено использовать библиотеки pandas, sklearn (кроме sklearn.metrics), xgboost и другие. 
Если scikit-learn (например, но и другие тоже) обернут в классы Transformer и Estimator, то их можно использовать.

Описание данных
Для выполнения работы вам следует взять все файлы из папки на HDFS /labs/slaba03/.

Давайте посмотрим, что у нас есть:

$ hdfs dfs -ls /labs/slaba03/
Found 4 items
-rw-r--r--   3 hdfs hdfs   91066524 2019-03-17 21:07 /labs/slaba03/laba03_items.csv
-rw-r--r--   3 hdfs hdfs   29965581 2019-03-17 21:07 /labs/slaba03/laba03_test.csv
-rw-r--r--   3 hdfs hdfs   74949368 2019-03-17 21:07 /labs/slaba03/laba03_train.csv
-rw-r--r--   3 hdfs hdfs  871302535 2019-03-17 21:07 /labs/slaba03/laba03_views_programmes.csv
В laba03_train.csv содержатся факты покупки (колонка purchase) пользователями (колонка user_id) телепередач (колонка item_id). 
Такой формат файла вам уже знаком.

laba03_items.csv — дополнительные данные по items. В данном файле много лишней или ненужной информации, так что задача её 
фильтрации и отбора ложится на вас. Поля в файле, на которых хотелось бы остановиться:

item_id — primary key. Соответствует item_id в предыдущем файле.
content_type — тип телепередачи (1 — платная, 0 — бесплатная). Вас интересуют платные передачи.
title — название передачи, текстовое поле.
year — год выпуска передачи, число.
genres — поле с жанрами передачи, разделёнными через запятую.

laba03_test.csv — тестовый датасет без указанного целевого признака purchase, который вам и предстоит предсказать.

Дополнительный файл laba03_views_programmes.csv по просмотрам передач с полями:
ts_start — время начала просмотра.
ts_end — время окончания просмотра.
item_type — тип просматриваемого контента:
live — просмотр "вживую", в момент показа контента в эфире.
pvr — просмотр в записи, после показа контента в эфире.

Результат
Предсказание целевой переменной "купит/не купит" — хорошо знакомая вам задача бинарной классификации. Поскольку нам важны именно
вероятности отнесения пары (пользователь, товар) к классу "купит" (1), то, на самом деле, вы можете подойти к проблеме с разных
сторон:

Как просто к задаче бинарной классификации. У вас есть два датасета, которые можно каким-то образом объединить, дополнительно 
обработать и сделать предсказания классификаторами (Spark ML).
Как к разработке рекомендательной системы: рекомендовать пользователю user_id топ-N лучших телепередач, которые были найдены по
    методике user-user / item-item коллаборативной фильтрации.
Как к задаче факторизации матриц: алгоритмы SVD, ALS, FM/FFM.
Советы
На качество прогноза в большей степени влияет качество признаков, которые вы сможете придумать из имеющихся данных, нежели выбор
и сложность алгоритма.
Качество входных данных также имеет сильное значение. Существует фраза "garbage in – garbage out". Мусор на входе – мусор на 
выходе. Потратьте время на подготовку и предобработку данных. Путь к успеху в третьей лабораторной:
    
Сосредоточьтесь на формировании следующих фичей: по файлу laba03_train.csv сформируйте признаки, характеризирующие как 
    интенсивно покупает пользователь и "покупаемость" item'ов
возьмите достаточно мощную модель (например GBTClassifier из pyspark'а)

Проверка
Эта лаба проходит в формате соревнования. Для вас оно начинается, когда вы успешно пройдёте минимальный порог — AUC должен 
составить не менее 0.79. После этого вы увидите лидерборд и сможете следить за результатами других участников.

Как уже было сказано, мы будем оценивать ваш алгоритм по метрике ROC AUC. Чекеру требуются вероятности в диапазоне [0.0, 1.0] 
отнесения пары (пользователь, товар) в тестовой выборке к классу "1" (купит).

Важно! Для точной проверки не забудьте отсортировать полученный файл по возрастанию идентификаторов пользователей (user_id), 
а затем — по возрастанию идентификаторов передач (item_id).

,user_id,item_id,purchase
0,1654,336,0.021805684684958027
1,1654,678,0.021805684684958027
2,1654,691,0.021805684684958027
3,1654,696,0.021805684684958027
...

In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as f

from pyspark import Row
import json
import re

conf = SparkConf()

spark.conf.set("spark.sql.crossJoin.enabled", True) # for cartesian product usage

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

In [3]:
schema = StructType(fields=[
    StructField('item_id', IntegerType()),
    StructField('user_id', IntegerType()),
    StructField('purchase',IntegerType())
])
df1 = spark.read.csv("hdfs:///labs/slaba03/laba03_train.csv",header='true',schema=schema)
df2 = spark.read.csv("hdfs:///labs/slaba03/laba03_test.csv", header='true',schema=schema)

In [4]:
users = df1.groupBy('user_id').mean('purchase').coalesce(10).cache()
items = df1.groupBy('item_id').mean('purchase').coalesce(10)

df2 = df2.join(users, on = 'user_id', how = 'outer').coalesce(10).withColumnRenamed('avg(purchase)', 'ave_user').cache()
df2 = df2.join(items, on = 'item_id', how = 'outer').coalesce(10).withColumnRenamed('avg(purchase)', 'ave_item').cache()

In [5]:
from pyspark.sql.window import Window

w1 = Window.partitionBy('user_id')
w2 = Window.partitionBy('item_id')

df1 = df1.withColumn('ave_user', f.avg('purchase').over(w1)) \
         .withColumn('ave_item', f.avg('purchase').over(w2)).coalesce(10).cache()

In [6]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import GBTClassifier
from pyspark.ml import Pipeline

vas = VectorAssembler(inputCols = ['ave_user', 'ave_item'], outputCol = 'features')
gbt = GBTClassifier(featuresCol = 'features', labelCol = 'purchase')
ppl = Pipeline(stages=[vas,gbt]).fit(df1)
predict = ppl.transform(df2)

In [7]:
from pyspark.sql.functions import udf

firstelement = udf(lambda v : float(v[1]), FloatType())
result = predict.select('user_id', 'item_id', firstelement('probability').alias('purchase')).coalesce(1).cache()
result = result.orderBy('user_id', 'item_id')

In [8]:
result_p = result.toPandas()
#result_p.to_csv('lab03.csv', index=False)

In [9]:
spark.stop()